# Assignment 4, task 3

In this task, you are going to implement a character model based on the Transformer architecture, starting from the provided skeleton. It is useful to first do exercise 2 before starting on this task.

The model you are going to implement here will have a context of 32 characters, i.e. it will consider the preceding 32 characters when estimating the probabilities of the possible character coming next. Due to the clever transformer architecture, the model will have less than 50,000 trainable parameters. As a comparison, the simpler model in exercise 2 only had a context of 8 characters but had more than 300,000 trainable parameters.


In [16]:
# First run this cell
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from datetime import datetime
import math

We need to map every type of input item (every character, in our case) to a unique ID number. Since we are not sure which characters will appear in our training text, we are going to create new IDs as we encounter new kinds of characters we haven't seen before.


In [17]:
char_to_id = {}  # Dictionary to store character-to-ID mapping
id_to_char = []  # List to store characters in their ID ordering
PADDING_SYMBOL = '<PAD>'
char_to_id[PADDING_SYMBOL] = 0
id_to_char.append(PADDING_SYMBOL)

We now define a class 'CharDataset' that extends the predefined 'Dataset' class.Compared to exercise 2, we will create data points in a slightly different way.

The init function reads a training text and splits it up into chunks $n$ characters long. From each chunk, $n$ data points with a corresponding label will be created, as in the following example:

Suppose $n=8$. From a chunk $[4,5,9,11,7,7,2,12]$ with 14 being the next character ID, the following data points and labels will be formed (0 is the padding symbol):

|          Data point | Label |
| ------------------: | ----: |
|   [4,0,0,0,0,0,0,0] |     5 |
|   [4,5,0,0,0,0,0,0] |     9 |
|   [4,5,9,0,0,0,0,0] |    11 |
|  [4,5,9,11,0,0,0,0] |     7 |
|  [4,5,9,11,7,0,0,0] |     7 |
|  [4,5,9,11,7,7,0,0] |     2 |
|  [4,5,9,11,7,7,2,0] |    12 |
| [4,5,9,11,7,7,2,12] |    14 |

This way, the model will learn to infer the next character even if the context is shorter than $n$. This is a very useful feature, particularly in 'real' language models, where the known context often is shorter than the maximal context length.


In [18]:
class CharDataset(Dataset):

    def __init__(self, file_path, n):
        self.datapoints = []
        self.labels = []
        chars = []
        try:
            # First read the dataset to find all the unique characters
            with open(file_path, 'r', encoding='utf-8') as f:
                contents = f.read()
            # YOUR CODE HERE
            for char in contents:
                if char not in char_to_id:
                    char_to_id[char] = len(id_to_char)
                    id_to_char.append(char)
                chars.append(char_to_id[char])
            # Then go through all the chars and chunk them up into datapoints
            k = 0
            while k < len(chars)-n:
                for i in range(1, n+1):
                    self.datapoints.append([c for c in chars[k:i+k]+[0]*(n-i)])
                    self.labels.append(chars[i+k])
                k += n
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"An error occurred: {e}")

    def __len__(self):
        return len(self.datapoints)

    def __getitem__(self, idx):
        idx = idx % len(self.datapoints)
        return torch.tensor(self.datapoints[idx]), torch.tensor(self.labels[idx], dtype=torch.long)

In [19]:
# Verify
# Max context is 32 characters long.
dataset = CharDataset('lang-eng-labs/exercise/HP_book_1.txt', 32)
d63, l63 = dataset[63]
d1048575, l1048575 = dataset[1048575]
d1048576, l1048576 = dataset[1048576]
print(d63[16].item() == l63.item())
print(d63[4].item() == l1048575.item())
print(sum(d1048576[1:]).item() == 0)

True
True
True


The **self-attention** computation is at the core of the Transformer architecture. It is important to get this computation efficient (i.e. vectorized), since it involves many matrix operations that would be very slow if implemented by Python loops.

The input to the self-attention computation is a tensor containing a vector for each input token, and the output is a tensor of the same dimensions, containing the contextualized versions of the input tokens (see Lecture 9 and the textbook, chapters 10.1 and 10.2).

Your task is to fill in the missing pieces below. Look for "REPLACE WITH YOUR CODE" and "YOUR CODE HERE".


In [32]:
class MultiHeadSelfAttention(nn.Module):
    """
    Calculates self-attention according to [Vaswani et al., NeurIPS, 2017]
    """

    def __init__(self, hidden_size, number_of_attention_heads):
        super().__init__()
        # The number of attention heads cannot be more than the hidden size
        assert hidden_size > number_of_attention_heads

        self.number_of_attention_heads = number_of_attention_heads
        # Divide the hidden_size roughly equal over the different heads
        self.attention_head_size = int(hidden_size / number_of_attention_heads)
        self.all_head_size = number_of_attention_heads * self.attention_head_size

        # Mapping from input to the query, key, and, value vectors
        self.query = nn.Linear(hidden_size, self.all_head_size, bias=False)
        self.key = nn.Linear(hidden_size, self.all_head_size, bias=False)
        self.value = nn.Linear(hidden_size, self.all_head_size, bias=False)

        self.final = nn.Linear(self.all_head_size, hidden_size, bias=False)

    def reshape_for_multihead_attention(self, x):
        # x has the shape (batch_size, seq_length, hidden_size)
        B, S, _ = x.shape

        # but we want to split the representation of each token into 'number_of_heads' parts:
        x = x.reshape(B, S, self.number_of_attention_heads,
                      self.attention_head_size)

        # and treat each part separately. Thus, we need the final tensor to have shape
        # (batch_size, number_of_heads, seq_length, attention_head_size)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states):
        # All of the tensors below will have the shape (batch_size, seq_length, hidden_size)
        query_all_heads = self.query(hidden_states)
        key_all_heads = self.key(hidden_states)
        value_all_heads = self.value(hidden_states)

        # All of the tensors below will have the shape (batch_size, number_of_heads, seq_length, attention_head_size)
        Q = self.reshape_for_multihead_attention(query_all_heads)
        K = self.reshape_for_multihead_attention(key_all_heads)
        V = self.reshape_for_multihead_attention(value_all_heads)

        # attention_scores will have the shape(batch_size, number_of_heads, seq_length, seq_length)
        attention_scores = torch.matmul(
            Q, K.transpose(-1, -2))  # REPLACE WITH YOUR CODE

        # Scale to reduce variance
        # REPLACE WITH YOUR CODE
        attention_scores /= math.sqrt(self.attention_head_size)

        # Use softmax to turn the attention scores into probabilities.
        # We want zero scores to be zero probabilities -- hence we turn
        # zero scores into -infinity before the softmax exponentiation.

        # YOUR CODE HERE
        attention_probs = nn.functional.softmax(
            attention_scores, dim=-1)  # REPLACE WITH YOUR CODE

        # Now produce the contextualized vectors for each head
        # The tensor below will have shape (batch_size, number_of_heads, seq_length, head_size)
        self_attention_all_heads_separately = torch.matmul(
            attention_probs, V)  # REPLACE WITH YOUR CODE

        # For each token, we now want to bring together the representation coming from each head.
        # The 'self_attention' tensor below should have the shape
        # (batch size, seq_length_, self.all_heads_size)
        self_attention = self_attention_all_heads_separately.permute(
            # REPLACE WITH YOUR CODE
            0, 2, 1, 3).reshape(-1, hidden_states.shape[1], self.all_head_size)

        # Finally, make sure that the output has the correct dimensions (batch_size,seq_length,hidden_size)
        return self.final(self_attention)

After the self-attention computation, the Transformer encoder block contains layer-normalization computations and a feed-forward layer. The code is given below.


In [33]:
class PositionwiseFFN(nn.Module):
    """
    The position-wise FFN that follows after the self-attention
    computation.
    """

    def __init__(self, hidden_size, dropout_prob):
        super().__init__()
        self.fc1 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.fc2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.dropout = nn.Dropout(dropout_prob)
        for module in (self.fc1, self.fc2):
            nn.init.kaiming_normal_(module.weight)
            nn.init.constant_(module.bias, 0.)

    def forward(self, x):
        return self.fc2(self.dropout(torch.relu(self.fc1(x))))


class EncoderBlock(nn.Module):
    """
    Transformer encoder block.

    This version differs from the original version in  [Vaswani et al. NeurIPS 2017],
    and applies the LayerNorm before the self-attention, and before the FFN, as this
    has proved to be beneficial (see [Nguyen and Salazar 2019]).
    """

    def __init__(self, hidden_size, number_of_attention_heads, dropout_prob):
        super().__init__()
        self.attn = MultiHeadSelfAttention(
            hidden_size, number_of_attention_heads)
        self.ffn = PositionwiseFFN(hidden_size, dropout_prob)
        self.dropout = nn.Dropout(dropout_prob)
        self.ln1 = nn.LayerNorm(hidden_size)
        self.ln2 = nn.LayerNorm(hidden_size)

    def forward(self, x):
        x1 = self.ln1(x)
        x2 = x + self.dropout(self.attn(x1))
        x3 = self.ln2(x2)
        x4 = x2 + self.dropout(self.ffn(x3))
        return x4

Here is the actual character-based language model, which uses the Transformer implementation above.


In [34]:
# ============= Hyper-parameters for training ============== #

class Config:
    number_of_transformer_encoders = 1
    number_of_attention_heads = 1
    hidden_size = 64
    dropout_prob = 0.1
    batch_size = 64
    learning_rate = 0.0003
    weight_decay = 0.000001
    no_of_epochs = 100


MAXLEN = 32   # This is the number of characters we will consider when
# predicting the next character

# ======================= The model ======================= #


class CharLM(nn.Module):

    def __init__(self, config, no_of_input_chars):
        super(CharLM, self).__init__()
        self.config = config
        self.embed = nn.Embedding(no_of_input_chars, config.hidden_size)
        # Make sure that the padding symbol (which has ID 0) is embedded
        # as a vector of 0s.
        self.embed.weight.data[0].fill_(0)
        self.positional = nn.Parameter(
            torch.randn(1, MAXLEN, config.hidden_size))
        modules = [EncoderBlock(config.hidden_size,
                                config.number_of_attention_heads,
                                config.dropout_prob) for _ in range(config.number_of_transformer_encoders)]
        self.transformers = nn.ModuleList(modules)
        self.final = nn.Linear(config.hidden_size*MAXLEN, no_of_input_chars)

    def forward(self, x):
        number_of_datapoints = x.shape[0]
        # First create a mask distinguishing 0 from positive word IDs
        non_zero_mask = (x != 0)
        word_embeddings = self.embed(x)
        # Add positional vectors in all non-padded positions
        pos = self.positional.expand_as(word_embeddings)
        pos = pos * non_zero_mask.unsqueeze(-1).float()
        t = word_embeddings + pos
        # Then apply the transformers and make a final prediction at the end
        for transf in self.transformers:
            t = transf(t)
        flattened_transf = t.reshape(number_of_datapoints, 1, -1)
        result = self.final(torch.tanh(flattened_transf))
        return result

In [35]:

# ======================= Training ======================= #

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Running on", device)

config = Config()
training_dataset = CharDataset('lang-eng-labs/exercise/HP_book_1.txt', 32)
print("There are", len(training_dataset), "datapoints and",
      len(id_to_char), "unique characters in the dataset")
training_loader = DataLoader(training_dataset, batch_size=config.batch_size)

charlm = CharLM(config, len(id_to_char)).to(device)
criterion = nn.CrossEntropyLoss()
charlm_optimizer = optim.Adam(charlm.parameters(), lr=config.learning_rate)

charlm.train()
print(datetime.now().strftime("%X"), "Training starts")
for epoch in range(config.no_of_epochs):
    iteration = 0
    for input_tensor, label in training_loader:
        input_tensor, label = input_tensor.to(device), label.to(device)
        charlm_optimizer.zero_grad()
        logits = charlm(input_tensor).to(device)
        loss = criterion(logits.squeeze(1), label)
        loss.backward()
        charlm_optimizer.step()
        iteration += 1

    print(datetime.now().strftime("%X"), "End of epoch",
          epoch+1, ", loss=", loss.detach().item())
    charlm.eval()
    # Generate 50 characters starting from the input text
    try:
        char_list = list("he took out his wand and"[-MAXLEN:])
        for i in range(300):
            input_tensor = torch.tensor([char_to_id[c] for c in char_list] + [
                                        char_to_id[PADDING_SYMBOL]]*(MAXLEN-len(char_list))).unsqueeze(0).to(device)
            logits = charlm(input_tensor).squeeze().to(device)
            _, new_character_tensor = logits.topk(1)
            new_character = id_to_char[new_character_tensor.detach().item()]
            print(new_character, end='')
            if len(char_list) == MAXLEN:
                char_list.pop(0)
            char_list.append(new_character)
        print()
    except KeyError:
        continue
    charlm.train()

Running on cuda
There are 442720 datapoints and 81 unique characters in the dataset
20:44:42 Training starts
20:44:59 End of epoch 1 , loss= 2.7797327041625977
  anng ana a the yele ana trin y ana and to o ana ulye y ana to the here a anury anld ge ary ana te the ana a tinng the a ana ye t ana there ye a anud and t he ana tey conu a foule to the here a ana sind ang ton he a ana uly to anule t to herere a any anan ging to onun a fut inin the a ana try a ano
20:45:17 End of epoch 2 , loss= 2.507356882095337
 se se anlel offere foully toun the ane anind s and here founly foully the tooure se san ane fone fone fon there fore st fully salle se saned saning to ang and the arin so sally se sallye sing se fone tone the the ane sing se foned an toun se sally sing the ane fine sering toung fing the an touly se
20:45:33 End of epoch 3 , loss= 2.2125518321990967
 sing sing anle se fonere stout y salle sally, ally, an an and the wand sand and off somery and sing of sered ane and so the fing sally f

In [37]:
# ==================== User interaction ==================== #

while True:
    text = input("> ").strip()
    if text == "":
        continue
    if text == "exit":
        break
    char_list = list(text[-MAXLEN:])
    # Generate 50 characters starting from the input text
    try:
        print(text, end='')
        for i in range(50):
            input_tensor = torch.tensor([char_to_id[c] for c in char_list] + [
                                        char_to_id[PADDING_SYMBOL]]*(MAXLEN-len(char_list))).unsqueeze(0).to(device)
            logits = charlm(input_tensor).squeeze().to(device)
            _, new_character_tensor = logits.topk(1)
            new_character = id_to_char[new_character_tensor.detach().item()]
            print(new_character, end='')
            if len(char_list) == MAXLEN:
                char_list.pop(0)
            char_list.append(new_character)
        print()
    except KeyError:
        continue

Harry stain, Madd Poter, I stan't looked the Gryffindor
Hermione and Harry, said Muggle some and the Gryffindore as the was 
Hermione stain, Madd Goy just. It all of the study them. 

The boy started and Malfor Malfoy, and and them. Gryffind
